# Set up
- install `osmnx`
- you don't need `osmnx` for the Adjust heading section

In [46]:
!echo $CONDA_DEFAULT_ENV

jlox


In [47]:
import sys
print(sys.executable)

/home/jlee4338/anaconda3/envs/jlox/bin/python


In [48]:
# basics
import os
import json
import yaml
import shutil
import zipfile
import random
import numpy as np
import pandas as pd
import json
import re
import requests

# network
import osmnx as ox
import networkx as nx

# spatial
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

# vis
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns 

In [49]:
# os.makedirs("./seg", exist_ok=True)
# os.makedirs("./asset", exist_ok=True)
base_dir = "./seg"

# Experiment

In [5]:
# Moreland Ave NE
#33.7563361201411, -84.34909614397644
# Peachtree Battle Ave NW
# 33.821001231769536, -84.39137284169695

# Lenox Park Blvd NE
# 33.84728263280257, -84.34277998010337
lat = 33.84728263280257
lon = -84.34277998010337

# 10th Street
# 33.781507, -84.395115
# lat = 33.781507
# lon = -84.395115

COI_for_ox = tuple([lat, lon])   # coordinate of interest
COI = Point(lat, lon)

COI = gpd.GeoDataFrame({"id": [0]}, geometry=[Point(lon, lat)], crs="EPSG:4326")
COI

,id,geometry
0,0,POINT (-84.34278 33.84728)


In [6]:
ox.settings.log_console = True
useful_tags = ox.settings.useful_tags_way + ["divider", "median", "barrier", "dual_carriageway"]
#print(useful_tags)
ox.settings.useful_tags_way = useful_tags
#print(ox.settings.useful_tags_way)

cf = ('["highway"~"trunk|primary|secondary|tertiary|residential|unclassified"]')
#cf = ('["dual_carriageway"~"yes"]')
G = ox.graph_from_point(COI_for_ox, 
                      #network_type='drive',
                      network_type='all',
                      dist = 200,
                      dist_type = "bbox",
                      #simplify=True,
                      truncate_by_edge=True,
                      custom_filter=cf)

print(len(G))
print(G)

G = ox.bearing.add_edge_bearings(G)
G = ox.projection.project_graph(G, 
                            to_crs = None) # by default, project to an appropriate UTM zone
nodes, edges = ox.graph_to_gdfs(G)
print(G.graph["crs"])
COI = COI.to_crs(G.graph["crs"])
print(COI.crs)

edges = edges.to_crs(G.graph["crs"])
edges.sort_values("name").iloc[0:5]

26
MultiDiGraph with 26 nodes and 58 edges
EPSG:32616
EPSG:32616


osmid   access      highway  oneway  reversed  \
u          v          key                                                    
5428749750 68406962   0    9189420      NaN  residential   False     False   
68406962   5428749750 0    9189420      NaN  residential   False      True   
68151315   68297959   0    9176587  private  residential   False     False   
68297941   68297959   0    9176592  private  residential   False      True   
           68151317   0    9176592  private  residential   False     False   

                              length  \
u          v          key              
5428749750 68406962   0    14.106907   
68406962   5428749750 0    14.106907   
68151315   68297959   0    42.531385   
68297941   68297959   0    27.044690   
           68151317   0    11.303212   

                                                                    geometry  \
u          v          key                                                      
5428749750 68406962   0    LINESTRING (745738.033 3748738.384, 745746.88 ...   
68406962   5428749750 0    LINESTRING (745746.88 3748749.368, 745738.033 ...   
68151315   68297959   0    LINESTRING (745707.863 3748227.827, 745719.2 3...   
68297941   68297959   0    LINESTRING (745723.044 3748192.257, 745732.397...   
           68151317   0    LINESTRING (745723.044 3748192.257, 745711.995...   

                              bearing                       name lanes  \
u          v          key                                                
5428749750 68406962   0     40.199949                Arbor Trace   NaN   
68406962   5428749750 0    220.200004                Arbor Trace   NaN   
68151315   68297959   0    119.061187  Fairway Estates Northeast   NaN   
68297941   68297959   0     53.696550  Fairway Estates Northeast   NaN   
           68151317   0    258.653089  Fairway Estates Northeast   NaN   

                          maxspeed  
u          v          key           
5428749750 68406962   0        NaN  
68406962   5428749750 0        NaN  
68151315   68297959   0        NaN  
68297941   68297959   0        NaN  
           68151317   0        NaN

# Study area

In [8]:
input_dir = "./seg"
edges_studyarea = gpd.read_file(os.path.join(input_dir, "top_10_road_segments_v2.0.geojson"))
edges_studyarea = edges_studyarea[['name', 'highway', 'road_id', 'link_id', 
                                   #'length_m', 'num_lanes', 'lane_width_ft', 
                                   #'RouteId_GDOT', 'BeginPoint_GDOT', 'EndPoint_GDOT',
                                   'geometry']]
edges_studyarea["midpoint"] = edges_studyarea.geometry.apply(lambda line: line.interpolate(line.length / 2))
edges_studyarea.geometry = edges_studyarea["midpoint"]
edges_studyarea = edges_studyarea.drop("geometry", axis=1)
edges_studyarea

,name,highway,road_id,link_id,midpoint
0,Church Street,primary,primary_1,cluster1_primary_1_1,POINT (-84.44097 33.68377)
1,East Point Street,primary,primary_2,cluster1_primary_2_1,POINT (-84.44267 33.67641)
2,East Point Street,primary,primary_2,cluster1_primary_2_2,POINT (-84.44235 33.67756)
3,East Point Street,primary,primary_2,cluster1_primary_2_3,POINT (-84.44211 33.67842)
4,East Point Street,primary,primary_2,cluster1_primary_2_4,POINT (-84.44195 33.67897)
...,...,...,...,...,...
1426,Langston Drive Southwest,tertiary,tertiary_3,cluster9_tertiary_3_1,POINT (-84.40993 33.69533)
1427,Perkerson Road Southwest,tertiary,tertiary_4,cluster9_tertiary_4_2,POINT (-84.41771 33.69988)
1428,Perkerson Road Southwest,tertiary,tertiary_4,cluster9_tertiary_4_3,POINT (-84.41575 33.69803)
1429,Perkerson Road Southwest,tertiary,tertiary_4,cluster9_tertiary_4_4,POINT (-84.41416 33.69712)


In [13]:
edges_studyarea[edges_studyarea["link_id"] == "cluster1_secondary_4_3"].explore()

# Functions

In [35]:
#=== Download OSM Road Network ===
cf = ('["highway"~"trunk|primary|secondary|tertiary|residential|unclassified"]')

#=== Determine Single / Dual Carriageway and Add Relevant Attributes (Median, Num. of Lanes) ===
def angular_diff(a, b):
    """Smallest difference between two angles (0–360)."""
    return min(abs(a - b), 360 - abs(a - b))

def has_opposite_bearings(series, threshold=150):
    bearings = series.dropna().to_numpy()
    n = len(bearings)
    for i in range(n):
        for j in range(i+1, n):
            if angular_diff(bearings[i], bearings[j]) > threshold:
                return True
    return False

def collapse_names(name):
    if isinstance(name, (list, tuple)):
        return str(name[0])
    return name
    
def collapse_lanes(val, how="min"):
    if isinstance(val, (list, tuple)):
        return min(int(x) for x in val if pd.notna(x))
    if pd.notna(val):
        return int(val)
    return pd.NA
    
#=== Find the Nearest Edge ===

#=== Cut Edge into 60m-long Segment ===

#=== Find PanoID Points ===
def sample_line(line, interval=5):
    return [line.interpolate(distance) for distance in range(0, int(line.length), interval)]
    
def _make_url(lat: float, lon: float) -> str:
    return (
        "https://maps.googleapis.com/maps/api/js/GeoPhotoService.SingleImageSearch"
        "?pb=!1m5!1sapiv3!5sUS!11m2!1m1!1b0!2m4!1m2!3d{lat}!4d{lon}!2d50!3m10"
        "!2m2!1sen!2sGB!9m1!1e2!11m4!1m3!1e2!2b1!3e2!4m10!1e1!1e2!1e3!1e4!1e8!1e6!5m1!1e2!6m1!1e2"
        "&callback=callbackfunc"
    ).format(lat=lat, lon=lon)

def _fetch(lat: float, lon: float, timeout: float = 10.0) -> str:
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "*/*",
        "Referer": "https://maps.google.com/",
    }
    r = requests.get(_make_url(lat, lon), headers=headers, timeout=timeout)
    r.raise_for_status()
    return r.text

def _strip_jsonp(text: str, cb="callbackfunc") -> str:
    m = re.search(rf"{re.escape(cb)}\s*\(\s*(.*)\s*\)\s*;?\s*$", text, re.DOTALL)
    if not m:
        raise ValueError("Could not find JSON payload inside JSONP wrapper.")
    return m.group(1)

def streetview_nearest_df(lat: float, lon: float) -> pd.DataFrame:
    """Return a DataFrame with columns: pano_id, lat, lon, heading, date."""
    payload = _strip_jsonp(_fetch(lat, lon))
    data = json.loads(payload)
    if data == [[5, "generic", "Search returned no images."]]:
        return pd.DataFrame(columns=["pano_id", "lat", "lon", "heading", "date"])
    subset = data[1][5][0]
    raw_panos = subset[3][0]
    raw_dates = [] if (len(subset) < 9 or subset[8] is None) else subset[8]
    raw_panos = raw_panos[::-1]
    raw_dates = raw_dates[::-1]
    dates = []
    for d in raw_dates:
        try:
            dates.append(f"{d[1][0]}-{int(d[1][1]):02d}")  # YYYY-MM
        except Exception:
            dates.append(None)
    rows = []
    for i, p in enumerate(raw_panos):
        try:
            pano_id = p[0][1]
            plat = float(p[2][0][2])
            plon = float(p[2][0][3])
            heading = float(p[2][2][0])
        except Exception:
            continue
        rows.append({
            "pano_id": pano_id,
            "pano_lat": plat,
            "pano_lon": plon,
            "pano_heading": heading,
            "pano_date": dates[i] if i < len(dates) else None
        })
    return pd.DataFrame(rows, columns=["pano_id", "pano_lat", "pano_lon", "pano_heading", "pano_date"])

recent_years = ["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024", "2025"]

def update_geometry(row):
    return Point(row["pano_lon"], row["pano_lat"])

# Run

In [41]:
def get_nearest_segmented_gsv_panos(
    COI_for_ox: tuple,
    COI: gpd.GeoDataFrame,
    LINK_ID: str,
    dist: int = 200,
    cut_length: float = 60,
    buffer_size: float = 30,
    sample_interval: int = 5,
    recent_years: list = None,
):
    """
    Given a center point (GeoDataFrame with 1 row), return segmented edge(s) 
    and deduplicated GSV pano points with attributes.

    Parameters
    ----------
    COI : GeoDataFrame
        A single-point GeoDataFrame in EPSG:4326.
    cf : str
        OSMnx custom_filter string (for filtering highways).
    dist : int, default=200
        Distance (in meters) to search for network around the COI.
    cut_length : float, default=60
        Length (in meters) of road segment to cut around the COI.
    buffer_size : float, default=30
        Buffer radius for cutting the road segment.
    sample_interval : int, default=5
        Interval (in meters) for sampling points along the road segment.
    recent_years : list, optional
        List of allowed pano years as strings. If None, defaults to 2016–2025.

    Returns
    -------
    edges_nearest_segmented : GeoDataFrame
        Segmented road edge(s) near the COI (in EPSG:4326).
    edges_nearest_segmented_gsv_pano_dedup : GeoDataFrame
        Deduplicated pano points with attributes (in EPSG:4326).
    """
    
    # === fixed parameters ===
    recent_years = [str(y) for y in range(2016, 2026)]
    cf = '["highway"~"trunk|primary|secondary|tertiary|residential|unclassified"]'

    
    # === Download OSM Road Network ===
    try:
        G = ox.graph_from_point(
            COI_for_ox, 
            network_type="all",
            dist=dist,
            dist_type="bbox",
            truncate_by_edge=True,
            custom_filter=cf)
    except ValueError as e:
        print(f"Skipping LINK_ID={LINK_ID}: {e}")
        return None, None
    
    G = ox.bearing.add_edge_bearings(G)
    G = ox.projection.project_graph(G, to_crs=None)
    CRS_UTM = G.graph["crs"]

    nodes, edges = ox.graph_to_gdfs(G)
    COI = COI.to_crs(CRS_UTM)
    edges = edges.to_crs(CRS_UTM)
    edges["dist_to_pt"] = edges.geometry.distance(COI.geometry.iloc[0])
    a = len(edges)
    
    # === Find nearest edges ===
    edges_nearest = edges.sort_values("dist_to_pt").iloc[0:2]
    #edges_nearest["name"] = edges_nearest["name"].astype(str)
    edges_nearest["name"] = edges_nearest["name"].apply(collapse_names)
    b = len(edges_nearest)
    #first_edge_name = edges_nearest["name"].iloc[0] # names are not consistent (e.g., Jr and Jr.)
    #print(first_edge_name)
    #edges_nearest = edges_nearest[edges_nearest["name"] == first_edge_name]

    # === Determine Single / Dual Carriageway and Add Relevant Attributes (Median, Num. of Lanes) ===
    #edges_nearest["has_opposite"] = edges_nearest.groupby("name")["bearing"].transform(lambda s: has_opposite_bearings(s)) # don't group by name!!
    edges_nearest["has_opposite"] = has_opposite_bearings(edges_nearest["bearing"])
    #print(edges_nearest)
    
    # Check conditions step by step
    if len(edges_nearest) > 1:
        both_oneway = (edges_nearest["oneway"].astype(str).iloc[0] == "True") & (edges_nearest["oneway"].astype(str).iloc[1] == "True")
        #same_name = edges_nearest["name"].iloc[0] == edges_nearest["name"].iloc[1] # names are not consistent (e.g., Jr and Jr.)
        opposite_bearings = edges_nearest["has_opposite"].iloc[0] & edges_nearest["has_opposite"].iloc[1]
        #if both_oneway and same_name and opposite_bearings:
        #print(f"both_oneway: {both_oneway}")
        #print(f"opposite_bearings: {opposite_bearings}")
        if both_oneway and opposite_bearings:
            # Dual carriageway -> assign median
            edges_nearest["median"] = "yes"
            edges_nearest = edges_nearest.iloc[0:2]
        else: 
            edges_nearest["median"] = "no"
            edges_nearest = edges_nearest.iloc[0]
            edges_nearest = gpd.GeoDataFrame(pd.DataFrame([edges_nearest]), crs=CRS_UTM)
    else:
        # Single carriageway -> no median
        edges_nearest["median"] = "no"
        edges_nearest = edges_nearest.iloc[0]
        edges_nearest = gpd.GeoDataFrame(pd.DataFrame([edges_nearest]), crs=CRS_UTM)
    c = len(edges_nearest)

    # Collapse lanes and the "name"
    if "lanes" in edges_nearest.columns:
        edges_nearest["lanes_collapsed"] = edges_nearest["lanes"].apply(collapse_lanes, how="min").astype("Int64")
    else:
        edges_nearest["lanes_collapsed"] = 1
    
    if len(edges_nearest) > 1:
        edges_nearest["lanes_collapsed"] = edges_nearest["lanes_collapsed"].sum()
        edges_nearest["name"] = str(edges_nearest.iloc[0]["name"])
    edges_nearest = edges_nearest.drop(["access", "reversed", "maxspeed"], axis=1, errors="ignore")

    # === Cut edge into segment ===
    COI_geom = COI["geometry"].iloc[0]
    edges_nearest["point_on_line"] = edges_nearest.apply(
        lambda row: row.geometry.interpolate(row.geometry.project(COI_geom)).buffer(buffer_size), axis=1
    )
    edges_nearest["geom_cut"] = edges_nearest.apply(
        lambda row: row.geometry.intersection(row.point_on_line), axis=1
    )
    edges_nearest_segmented = gpd.GeoDataFrame(edges_nearest, geometry=edges_nearest["geom_cut"]).drop(
        columns=["point_on_line", "geom_cut"]
    )
    edges_nearest_segmented["length_new"] = edges_nearest_segmented.geometry.length
    edges_nearest_segmented = edges_nearest_segmented.reset_index(drop=True)
    edges_nearest_segmented["link_id"] = LINK_ID
    edges_nearest_segmented = edges_nearest_segmented[['link_id', 'osmid', 'name', 'highway', 'oneway', 
                                                       'bearing', 'median', 'lanes_collapsed',
                                                       'length_new', 'geometry']]
    edges_nearest_segmented = edges_nearest_segmented.rename(columns={"highway": "road_type"})
    d = len(edges_nearest_segmented)
    print(f"edges: {a} -> {b} -> {c} -> {d}")

    # === Sample points ===
    edges_gsv = edges_nearest_segmented.copy()
    edges_gsv["osmid"] = edges_gsv["osmid"].astype(str)
    edges_gsv = edges_gsv[["osmid", "oneway", "median", "bearing", "name", "geometry"]]
    edges_gsv["sampled_points"] = edges_gsv.geometry.apply(lambda x: sample_line(x, interval=sample_interval))
    edges_gsv = edges_gsv.explode("sampled_points")
    edges_gsv = gpd.GeoDataFrame(edges_gsv, geometry="sampled_points", crs=CRS_UTM)
    edges_gsv = edges_gsv.to_crs(4326)
    edges_gsv["lat"] = edges_gsv.geometry.y
    edges_gsv["lon"] = edges_gsv.geometry.x
    
    # === Query GSV ===
    results = []
    for _, row in edges_gsv.iterrows():
        out = streetview_nearest_df(row.lat, row.lon)
        out["osmid"] = row["osmid"]
        out["bearing"] = row["bearing"]
        out["median"] = row["median"]
        results.append(out)
    pano_df = pd.concat(results, ignore_index=True)
    if pano_df.empty:
        return None, None   # or pd.DataFrame(), pd.DataFrame()
    #print(pano_df)
    
    pano_df = pano_df[~pano_df["pano_date"].isna()]
    a = len(pano_df)
    if pano_df.empty:
        return None, None   # or pd.DataFrame(), pd.DataFrame()
        
    # Filter by recent years
    pano_df = pano_df[pano_df["pano_date"].apply(lambda x: x.split("-")[0]).isin(recent_years)]
    b = len(pano_df) # 39
    if pano_df.empty:
        return None, None   # or pd.DataFrame(), pd.DataFrame()

    # Mode year filtering
    pano_df["count"] = pano_df.groupby(["osmid", "pano_date"])["pano_date"].transform("count")
    pano_df["max_count"] = pano_df.groupby("osmid")["count"].transform("max")
    pano_df = pano_df[pano_df["count"] == pano_df["max_count"]]

    if pano_df.empty:
        return None, None   # or pd.DataFrame(), pd.DataFrame()
    c = len(pano_df) #12
    if pano_df.empty:
        return None, None   # or pd.DataFrame(), pd.DataFrame()

    # Deduplicate
    pano_df = pano_df.drop_duplicates(["osmid", "pano_id"])
    d = len(pano_df) #6
    pano_df["point_id"] = pano_df.groupby("osmid").cumcount()
    #pano_df["count"] = pano_df.groupby("pano_date")["pano_date"].transform("count") # update the count info after de-dup
    pano_df["geometry"] = pano_df.apply(update_geometry, axis=1)

    pano_df["link_id"] = LINK_ID
    pano_df = pano_df[['link_id', 'osmid', 'median', 'bearing', 'pano_id', 'pano_lat', 'pano_lon', 'pano_heading', 'pano_date', 'point_id', 'geometry']]

    edges_nearest_segmented_gsv_pano_dedup = gpd.GeoDataFrame(pano_df, geometry="geometry", crs="EPSG:4326")

    e = len(pano_df) #6
    print(f"points: {a} -> {b} -> {c} -> {d} -> {e}")
    #print()
    
    edges_nearest_segmented = edges_nearest_segmented.to_crs(4326)

    return edges_nearest_segmented, edges_nearest_segmented_gsv_pano_dedup

In [42]:
results_edges = []
results_points = []

#for i, row in edges_studyarea.iloc[75:77].iterrows():
#for i, row in edges_studyarea.iloc[856:].iterrows():  
for i, row in edges_studyarea[edges_studyarea["link_id"] == "cluster1_secondary_4_3"].iterrows():
#for i, row in edges_studyarea.iterrows(): 
    print(f"{i}: {row["name"]}")
    COI_geom = row["midpoint"]
    COI = gpd.GeoDataFrame(geometry=[COI_geom], crs="EPSG:4326")

    lon = COI_geom.x
    lat = COI_geom.y
    COI_for_ox = tuple([lat, lon]) 

    link_id = row["link_id"]

    output_edge, output_point = get_nearest_segmented_gsv_panos(COI_for_ox, COI, link_id)
    results_edges.append(output_edge)
    results_points.append(output_point)
    print()
#results 

38: Headland Drive
edges: 54 -> 2 -> 2 -> 2
points: 51 -> 39 -> 14 -> 7 -> 7



In [43]:
results_edges = pd.concat(results_edges)
print(len(results_edges))
results_edges.iloc[0:5]
#results_edges.explore()

2


,link_id,osmid,name,road_type,oneway,bearing,median,lanes_collapsed,length_new,geometry
0,cluster1_secondary_4_3,"[1072281837, 146799870]",Headland Drive,secondary,True,249.713610,yes,3,59.929328,"LINESTRING (-84.44842 33.68559, -84.44901 33.6..."
1,cluster1_secondary_4_3,"[1072229296, 569629630]",Headland Drive,secondary,True,69.298313,yes,3,59.964709,"LINESTRING (-84.44895 33.68526, -84.44884 33.6..."


In [44]:
results_points = pd.concat(results_points)
print(len(results_points))
results_points.iloc[0:5]
results_points.explore()

7


# Assign Side 1 / 2

## export

In [45]:
# save_dir = "./output"

# results_edges.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0_2.geojson"), driver="GeoJSON")
# results_points.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0_pano_2.geojson"), driver="GeoJSON")

# #m.save(os.path.join(save_dir, "top_10_road_segments_v3.0.html"))

## import

In [25]:
input_dir = "./output"
edge_0 = os.path.join(input_dir, "top_10_road_segments_v3.0_0.geojson")
edge_1 = os.path.join(input_dir, "top_10_road_segments_v3.0_1.geojson")
edge_2 = os.path.join(input_dir, "top_10_road_segments_v3.0_2.geojson")

edge_0 = gpd.read_file(edge_0)
edge_1 = gpd.read_file(edge_1)
edge_2 = gpd.read_file(edge_2)

edge_whole = pd.concat([edge_0, edge_1, edge_2], axis=0).sort_values(["link_id", "osmid"])
print(len(edge_whole))
edge_whole.iloc[0:5]

Skipping field highway: unsupported OGR type: 5
Skipping field highway: unsupported OGR type: 5
Skipping field highway: unsupported OGR type: 5


1411


,link_id,osmid,name,oneway,bearing,median,lanes_collapsed,length_new,geometry
101,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",Campbellton Road,False,122.951365,no,4.0,59.962947,"LINESTRING (-84.51693 33.69262, -84.51678 33.6..."
102,cluster10_primary_1_3,1145035107,Campbellton Road,False,304.354453,no,5.0,59.961066,"LINESTRING (-84.51508 33.69156, -84.51558 33.6..."
103,cluster10_primary_1_5,1145035107,Campbellton Road,False,294.081106,no,5.0,59.976616,"LINESTRING (-84.51373 33.69093, -84.51398 33.6..."
104,cluster10_primary_1_6,"[ 1145035112, 1145035113, 1145035108, 11450351...",Campbellton Road,False,97.915169,no,5.0,59.983905,"LINESTRING (-84.51277 33.69076, -84.51271 33.6..."
105,cluster10_tertiary_2_1,"[ 9261978, 230251492 ]",Daniel Road Southwest,False,310.554014,no,NaN,59.953501,"LINESTRING (-84.51268 33.68657, -84.51295 33.6..."


In [24]:
point_0 = os.path.join(input_dir, "top_10_road_segments_v3.0_pano_0.geojson")
point_1 = os.path.join(input_dir, "top_10_road_segments_v3.0_pano_1.geojson")
point_2 = os.path.join(input_dir, "top_10_road_segments_v3.0_pano_2.geojson")

point_0 = gpd.read_file(point_0)
point_1 = gpd.read_file(point_1)
point_2 = gpd.read_file(point_2)

point_whole = pd.concat([point_0, point_1, point_2], axis=0).sort_values(["link_id", "osmid", "point_id"])
print(len(point_whole))
point_whole.iloc[0:5]

8741


,link_id,osmid,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,point_id,is_midpoint,geometry
622,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,0,yes,POINT (-84.51693 33.69266)
623,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,1,no,POINT (-84.51683 33.6926)
624,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,2,no,POINT (-84.51673 33.69255)
625,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,3,yes,POINT (-84.51662 33.69249)
626,cluster10_primary_1_2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,4,no,POINT (-84.51652 33.69243)


## edges

In [41]:
edge_whole_single = edge_whole[edge_whole["median"] == "no"]
print(len(edge_whole_single))
print()
edge_whole_single_side1 = edge_whole_single.copy()
edge_whole_single_side2 = edge_whole_single.copy()
print(len(edge_whole_single_side1))
edge_whole_single_side1["side"] = "side1"
print(edge_whole_single_side1.iloc[0:2])
print()
print(len(edge_whole_single_side2))
edge_whole_single_side2["side"] = "side2"
edge_whole_single_side2["bearing"] = (edge_whole_single_side2["bearing"] + 180) % 360
print(edge_whole_single_side2.iloc[0:2])

1335

1335
                   link_id                                              osmid  \
101  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...   
102  cluster10_primary_1_3                                         1145035107   

                 name  oneway     bearing median  lanes_collapsed  length_new  \
101  Campbellton Road   False  122.951365     no              4.0   59.962947   
102  Campbellton Road   False  304.354453     no              5.0   59.961066   

                                              geometry   side  
101  LINESTRING (-84.51693 33.69262, -84.51678 33.6...  side1  
102  LINESTRING (-84.51508 33.69156, -84.51558 33.6...  side1  

1335
                   link_id                                              osmid  \
101  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...   
102  cluster10_primary_1_3                                         1145035107   

                 name  oneway     bearing median  lanes_col

In [42]:
edge_whole_dual = edge_whole[edge_whole["median"] == "yes"]
print(len(edge_whole_dual))

def compare_bearing(g):
    min_bearing = g["bearing"].min()
    return g["bearing"] == min_bearing

edge_whole_dual["bearing_smaller"] = edge_whole_dual.groupby("link_id")["bearing"].transform(lambda g: g == g.min()) 
edge_whole_dual["side"] = edge_whole_dual["bearing_smaller"].map(
    {True: "side1", False: "side2"}
)
edge_whole_dual.drop("bearing_smaller", axis=1, inplace=True)
print(len(edge_whole_dual))
edge_whole_dual.iloc[0:6]

76
76


/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/tmp/ipykernel_782598/1238855388.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

,link_id,osmid,name,oneway,bearing,median,lanes_collapsed,length_new,geometry,side
116,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",Cook Road,True,99.726642,yes,4.0,60.061208,"LINESTRING (-84.30891 33.71502, -84.30873 33.7...",side1
115,cluster13_secondary_2_1,"[ 9167493, 989374654 ]",Cook Road,True,283.830244,yes,4.0,60.019951,"LINESTRING (-84.30798 33.71562, -84.30801 33.7...",side2
135,cluster14_secondary_2_1,1029615999,Glenridge Drive Northeast,True,156.926367,yes,4.0,58.801198,"LINESTRING (-84.36346 33.91803, -84.36321 33.9...",side1
136,cluster14_secondary_2_1,"[ 946371847, 1029631250, 1029631251, 102963125...",Glenridge Drive Northeast,True,346.088643,yes,4.0,59.943976,"LINESTRING (-84.36311 33.91757, -84.36311 33.9...",side2
137,cluster14_secondary_2_2,1029615999,Glenridge Drive Northeast,True,153.016302,yes,4.0,59.967246,"LINESTRING (-84.36385 33.91875, -84.36379 33.9...",side1
138,cluster14_secondary_2_2,"[ 946371847, 1029631250, 1029631251, 102963125...",Glenridge Drive Northeast,True,346.088643,yes,4.0,57.733423,"LINESTRING (-84.36348 33.9183, -84.36355 33.91...",side2


In [47]:
edge_whole_side_assigned = pd.concat([edge_whole_single_side1, edge_whole_single_side2, edge_whole_dual], axis=0)
edge_whole_side_assigned = edge_whole_side_assigned.sort_values(["link_id", "side"])
edge_whole_side_assigned = edge_whole_side_assigned[["link_id", "side", "name", "osmid", "oneway", "median", "bearing", "lanes_collapsed", "length_new", "geometry"]]
print(len(edge_whole_side_assigned))
edge_whole_side_assigned.iloc[0:10]
#edge_whole_side_assigned.iloc[-10:]
#edge_whole_side_assigned[edge_whole_side_assigned["median"]=="yes"]

2746


,link_id,side,name,osmid,oneway,median,bearing,lanes_collapsed,length_new,geometry
101,cluster10_primary_1_2,side1,Campbellton Road,"[ 1145035104, 1145035105, 1145035106, 20726027...",False,no,122.951365,4.0,59.962947,"LINESTRING (-84.51693 33.69262, -84.51678 33.6..."
101,cluster10_primary_1_2,side2,Campbellton Road,"[ 1145035104, 1145035105, 1145035106, 20726027...",False,no,302.951365,4.0,59.962947,"LINESTRING (-84.51693 33.69262, -84.51678 33.6..."
102,cluster10_primary_1_3,side1,Campbellton Road,1145035107,False,no,304.354453,5.0,59.961066,"LINESTRING (-84.51508 33.69156, -84.51558 33.6..."
102,cluster10_primary_1_3,side2,Campbellton Road,1145035107,False,no,124.354453,5.0,59.961066,"LINESTRING (-84.51508 33.69156, -84.51558 33.6..."
103,cluster10_primary_1_5,side1,Campbellton Road,1145035107,False,no,294.081106,5.0,59.976616,"LINESTRING (-84.51373 33.69093, -84.51398 33.6..."
103,cluster10_primary_1_5,side2,Campbellton Road,1145035107,False,no,114.081106,5.0,59.976616,"LINESTRING (-84.51373 33.69093, -84.51398 33.6..."
104,cluster10_primary_1_6,side1,Campbellton Road,"[ 1145035112, 1145035113, 1145035108, 11450351...",False,no,97.915169,5.0,59.983905,"LINESTRING (-84.51277 33.69076, -84.51271 33.6..."
104,cluster10_primary_1_6,side2,Campbellton Road,"[ 1145035112, 1145035113, 1145035108, 11450351...",False,no,277.915169,5.0,59.983905,"LINESTRING (-84.51277 33.69076, -84.51271 33.6..."
105,cluster10_tertiary_2_1,side1,Daniel Road Southwest,"[ 9261978, 230251492 ]",False,no,310.554014,NaN,59.953501,"LINESTRING (-84.51268 33.68657, -84.51295 33.6..."
105,cluster10_tertiary_2_1,side2,Daniel Road Southwest,"[ 9261978, 230251492 ]",False,no,130.554014,NaN,59.953501,"LINESTRING (-84.51268 33.68657, -84.51295 33.6..."


## points

In [50]:
point_whole_single = point_whole[point_whole["median"] == "no"]
print(len(point_whole_single))
print()
point_whole_single_side1 = point_whole_single.copy()
point_whole_single_side2 = point_whole_single.copy()
print(len(point_whole_single_side1))
point_whole_single_side1["side"] = "side1"
print(point_whole_single_side1.iloc[0:2])
print()
print(len(point_whole_single_side2))
point_whole_single_side2["side"] = "side2"
point_whole_single_side2["bearing"] = (point_whole_single_side2["bearing"] + 180) % 360
print(point_whole_single_side2.iloc[0:2])

8275

8275
                   link_id                                              osmid  \
622  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...   
623  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...   

    median     bearing                 pano_id   pano_lat   pano_lon  \
622     no  122.951365  L0BLSeHmcehpH9rKz6nM0Q  33.692660 -84.516929   
623     no  122.951365  piW2JSC7o-e25SiBjEFtlw  33.692603 -84.516828   

     pano_heading pano_date  point_id is_midpoint                    geometry  \
622    303.943359   2024-06         0         yes  POINT (-84.51693 33.69266)   
623    303.892731   2024-06         1          no   POINT (-84.51683 33.6926)   

      side  
622  side1  
623  side1  

8275
                   link_id                                              osmid  \
622  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...   
623  cluster10_primary_1_2  [ 1145035104, 1145035105, 1145035106, 20726027...

In [51]:
point_whole_dual = point_whole[point_whole["median"] == "yes"]
print(len(point_whole_dual))

def compare_bearing(g):
    min_bearing = g["bearing"].min()
    return g["bearing"] == min_bearing

point_whole_dual["bearing_smaller"] = point_whole_dual.groupby("link_id")["bearing"].transform(lambda g: g == g.min()) 
point_whole_dual["side"] = point_whole_dual["bearing_smaller"].map(
    {True: "side1", False: "side2"}
)
point_whole_dual.drop("bearing_smaller", axis=1, inplace=True)
print(len(point_whole_dual))
point_whole_dual.iloc[0:6]

466
466


/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/tmp/ipykernel_782598/2819407075.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

,link_id,osmid,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,point_id,is_midpoint,geometry,side
716,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,OaV-jqmUlsx2zVO0Uq512Q,33.714983,-84.308894,114.411766,2018-11,0,no,POINT (-84.30889 33.71498),side1
717,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,kiHpXyj5UUS2-V0zEu0I4Q,33.714942,-84.308777,112.652451,2018-11,1,no,POINT (-84.30878 33.71494),side1
718,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,gKmMYaVEu6JaJ3bujBT_aA,33.714902,-84.308656,110.505569,2018-11,2,no,POINT (-84.30866 33.7149),side1
719,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,rShAMGFsJN12B08ov4B0pA,33.714883,-84.308594,108.882721,2018-11,3,yes,POINT (-84.30859 33.71488),side1
720,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,_YeLhIkQaxQ-aS06MeOBsQ,33.714840,-84.308448,107.129433,2018-11,4,no,POINT (-84.30845 33.71484),side1
721,cluster13_secondary_2_1,"[ 29344713, 949955811, 949955812 ]",yes,99.726642,QHejjs9mcInLQsORL1Ae6w,33.714816,-84.308343,104.320358,2018-11,5,no,POINT (-84.30834 33.71482),side1


In [58]:
point_whole_side_assigned = pd.concat([point_whole_single_side1, point_whole_single_side2, point_whole_dual], axis=0)
point_whole_side_assigned = point_whole_side_assigned.sort_values(["link_id", "side", "point_id"])
point_whole_side_assigned = point_whole_side_assigned[["link_id", "side", "point_id", "osmid", "median", "bearing", 
                                                       "pano_id", "pano_lat", "pano_lon", "pano_heading", "pano_date", "is_midpoint",
                                                       "geometry"]]
print(len(point_whole_side_assigned))
point_whole_side_assigned.iloc[0:10]
#point_whole_side_assigned.iloc[-10:]
#point_whole_side_assigned[point_whole_side_assigned["median"]=="yes"]

17016


,link_id,side,point_id,osmid,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
622,cluster10_primary_1_2,side1,0,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,yes,POINT (-84.51693 33.69266)
623,cluster10_primary_1_2,side1,1,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,no,POINT (-84.51683 33.6926)
624,cluster10_primary_1_2,side1,2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,no,POINT (-84.51673 33.69255)
625,cluster10_primary_1_2,side1,3,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,yes,POINT (-84.51662 33.69249)
626,cluster10_primary_1_2,side1,4,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,no,POINT (-84.51652 33.69243)
627,cluster10_primary_1_2,side1,5,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,aZ9A4vmlxUcUrHcWA00GJQ,33.692371,-84.516419,304.434784,2024-06,no,POINT (-84.51642 33.69237)
622,cluster10_primary_1_2,side2,0,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,yes,POINT (-84.51693 33.69266)
623,cluster10_primary_1_2,side2,1,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,no,POINT (-84.51683 33.6926)
624,cluster10_primary_1_2,side2,2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,no,POINT (-84.51673 33.69255)
625,cluster10_primary_1_2,side2,3,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,yes,POINT (-84.51662 33.69249)


In [61]:
def midpoint_flags(group):
    n = len(group)
    # decide which position is midpoint
    if n == 1:
        mid_pos = 0
    elif n % 2 == 1:   # odd
        mid_pos = n // 2
    else:              # even → the one right before middle
        mid_pos = n // 2 - 1

    flags = [False] * n
    flags[mid_pos] = True
    return pd.Series(flags, index=group.index)

# Apply group-wise and attach as new column
point_whole_side_assigned["is_midpoint"] = (
    point_whole_side_assigned.groupby(["link_id", "side"], group_keys=False)
    .apply(midpoint_flags)
)

print(len(point_whole_side_assigned))
point_whole_side_assigned.iloc[0:10]

17016


/tmp/ipykernel_782598/1238967038.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(midpoint_flags)


,link_id,side,point_id,osmid,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
622,cluster10_primary_1_2,side1,0,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (-84.51693 33.69266)
623,cluster10_primary_1_2,side1,1,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (-84.51683 33.6926)
624,cluster10_primary_1_2,side1,2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (-84.51673 33.69255)
625,cluster10_primary_1_2,side1,3,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (-84.51662 33.69249)
626,cluster10_primary_1_2,side1,4,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,False,POINT (-84.51652 33.69243)
627,cluster10_primary_1_2,side1,5,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,aZ9A4vmlxUcUrHcWA00GJQ,33.692371,-84.516419,304.434784,2024-06,False,POINT (-84.51642 33.69237)
622,cluster10_primary_1_2,side2,0,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (-84.51693 33.69266)
623,cluster10_primary_1_2,side2,1,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (-84.51683 33.6926)
624,cluster10_primary_1_2,side2,2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (-84.51673 33.69255)
625,cluster10_primary_1_2,side2,3,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,302.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (-84.51662 33.69249)


In [69]:
edge_whole_side_assigned.drop("osmid", axis=1).iloc[0:5]

,link_id,side,name,oneway,median,bearing,lanes_collapsed,length_new,geometry
101,cluster10_primary_1_2,side1,Campbellton Road,False,no,122.951365,4.0,59.962947,"LINESTRING (730156.397 3730842.752, 730170.819..."
101,cluster10_primary_1_2,side2,Campbellton Road,False,no,302.951365,4.0,59.962947,"LINESTRING (730156.397 3730842.752, 730170.819..."
102,cluster10_primary_1_3,side1,Campbellton Road,False,no,304.354453,5.0,59.961066,"LINESTRING (730331.116 3730729.807, 730284.049..."
102,cluster10_primary_1_3,side2,Campbellton Road,False,no,124.354453,5.0,59.961066,"LINESTRING (730331.116 3730729.807, 730284.049..."
103,cluster10_primary_1_5,side1,Campbellton Road,False,no,294.081106,5.0,59.976616,"LINESTRING (730458.195 3730662.287, 730434.873..."


In [68]:
point_whole_side_assigned.drop("osmid", axis=1).iloc[0:15]

,link_id,side,point_id,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
622,cluster10_primary_1_2,side1,0,no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (730156.637 3730847.262)
623,cluster10_primary_1_2,side1,1,no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (730166.158 3730841.151)
624,cluster10_primary_1_2,side1,2,no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (730175.836 3730834.989)
625,cluster10_primary_1_2,side1,3,no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (730185.5 3730828.809)
626,cluster10_primary_1_2,side1,4,no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,False,POINT (730195.089 3730822.63)
627,cluster10_primary_1_2,side1,5,no,122.951365,aZ9A4vmlxUcUrHcWA00GJQ,33.692371,-84.516419,304.434784,2024-06,False,POINT (730204.703 3730816.366)
622,cluster10_primary_1_2,side2,0,no,302.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (730156.637 3730847.262)
623,cluster10_primary_1_2,side2,1,no,302.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (730166.158 3730841.151)
624,cluster10_primary_1_2,side2,2,no,302.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (730175.836 3730834.989)
625,cluster10_primary_1_2,side2,3,no,302.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (730185.5 3730828.809)


## single carriageway case

In [77]:
LINK_ID = "cluster10_primary_1_2"
edge_whole_side_assigned[edge_whole_side_assigned["link_id"]==LINK_ID].drop("osmid", axis=1)

,link_id,side,name,oneway,median,bearing,lanes_collapsed,length_new,geometry
101,cluster10_primary_1_2,side1,Campbellton Road,False,no,122.951365,4.0,59.962947,"LINESTRING (730156.397 3730842.752, 730170.819..."
101,cluster10_primary_1_2,side2,Campbellton Road,False,no,302.951365,4.0,59.962947,"LINESTRING (730156.397 3730842.752, 730170.819..."


In [78]:
point_whole_side_assigned[point_whole_side_assigned["link_id"]==LINK_ID].drop("osmid", axis=1)

,link_id,side,point_id,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
622,cluster10_primary_1_2,side1,0,no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (730156.637 3730847.262)
623,cluster10_primary_1_2,side1,1,no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (730166.158 3730841.151)
624,cluster10_primary_1_2,side1,2,no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (730175.836 3730834.989)
625,cluster10_primary_1_2,side1,3,no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (730185.5 3730828.809)
626,cluster10_primary_1_2,side1,4,no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,False,POINT (730195.089 3730822.63)
627,cluster10_primary_1_2,side1,5,no,122.951365,aZ9A4vmlxUcUrHcWA00GJQ,33.692371,-84.516419,304.434784,2024-06,False,POINT (730204.703 3730816.366)
622,cluster10_primary_1_2,side2,0,no,302.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (730156.637 3730847.262)
623,cluster10_primary_1_2,side2,1,no,302.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (730166.158 3730841.151)
624,cluster10_primary_1_2,side2,2,no,302.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (730175.836 3730834.989)
625,cluster10_primary_1_2,side2,3,no,302.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (730185.5 3730828.809)


In [79]:
m = edge_whole_side_assigned[edge_whole_side_assigned["link_id"]==LINK_ID].explore()

m = point_whole_side_assigned[point_whole_side_assigned["link_id"]==LINK_ID].explore(
    m=m,
    color="red")
m

## dual carriageway case

In [76]:
LINK_ID = "cluster14_secondary_2_1"
edge_whole_side_assigned[edge_whole_side_assigned["link_id"]==LINK_ID].drop("osmid", axis=1)

,link_id,side,name,oneway,median,bearing,lanes_collapsed,length_new,geometry
135,cluster14_secondary_2_1,side1,Glenridge Drive Northeast,True,yes,156.926367,4.0,58.801198,"LINESTRING (743745.33 3756197.823, 743769.846 ..."
136,cluster14_secondary_2_1,side2,Glenridge Drive Northeast,True,yes,346.088643,4.0,59.943976,"LINESTRING (743778.736 3756147.428, 743778.531..."


In [74]:
point_whole_side_assigned[point_whole_side_assigned["link_id"]==LINK_ID].drop("osmid", axis=1)

,link_id,side,point_id,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
829,cluster14_secondary_2_1,side1,0,yes,156.926367,2UuJp3ghiIe9RZrPZ17a5Q,33.918023,-84.363433,156.800858,2019-10,False,POINT (743747.607 3756197.299)
830,cluster14_secondary_2_1,side1,1,yes,156.926367,5hglOp5kccvmkajTouZCwQ,33.917980,-84.363412,156.866409,2019-10,False,POINT (743749.671 3756192.66)
831,cluster14_secondary_2_1,side1,2,yes,156.926367,Le6PW1kWYs5KR21MSIvyOg,33.917895,-84.363370,156.792618,2019-10,False,POINT (743753.779 3756183.297)
832,cluster14_secondary_2_1,side1,3,yes,156.926367,_6S_5sqK5DXrDftlaSeRRQ,33.917810,-84.363328,156.525742,2019-10,True,POINT (743757.961 3756173.933)
833,cluster14_secondary_2_1,side1,4,yes,156.926367,C1LsSOh2CeRqZUgRQBZWOg,33.917693,-84.363267,155.998810,2019-10,False,POINT (743763.866 3756161.087)
834,cluster14_secondary_2_1,side1,5,yes,156.926367,VzPvGROuGbOZTpKKTGB5Wg,33.917606,-84.363222,155.885254,2019-10,False,POINT (743768.304 3756151.583)
835,cluster14_secondary_2_1,side1,6,yes,156.926367,dgf5VqczMdnKqNmiSdYfBg,33.917565,-84.363203,156.106216,2019-10,False,POINT (743770.193 3756147.128)
836,cluster14_secondary_2_1,side2,0,yes,346.088643,LqW46649JFgZ686tJ4Jf0A,33.917578,-84.363090,333.416626,2019-10,False,POINT (743780.557 3756148.782)
837,cluster14_secondary_2_1,side2,1,yes,346.088643,pvjxW_qbIgeMDAitjdLRfQ,33.917658,-84.363134,335.519135,2019-10,False,POINT (743776.286 3756157.585)
838,cluster14_secondary_2_1,side2,2,yes,346.088643,4CvyUV-QwY5KtjzC-JQv9w,33.917739,-84.363176,336.362854,2019-10,True,POINT (743772.22 3756166.438)


In [45]:
# m = edge_whole_side_assigned[edge_whole_side_assigned["link_id"]==LINK_ID].explore()

# m = point_whole_side_assigned[point_whole_side_assigned["link_id"]==LINK_ID].explore(
#     m=m,
#     color="red")

# Export

In [62]:
m = edge_whole_side_assigned.explore()

m = point_whole_side_assigned.explore(
    m=m,
    color="red")

In [63]:
save_dir = "./output"

edge_whole_side_assigned = edge_whole_side_assigned.to_crs(4326)
point_whole_side_assigned = point_whole_side_assigned.to_crs(4326)
edge_whole_side_assigned.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0.geojson"), driver="GeoJSON")
point_whole_side_assigned.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0_pano.geojson"), driver="GeoJSON")

m.save(os.path.join(save_dir, "top_10_road_segments_v3.0.html"))

edge_whole_side_assigned = edge_whole_side_assigned.to_crs(32616)
point_whole_side_assigned = point_whole_side_assigned.to_crs(32616)
edge_whole_side_assigned.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0_32616.geojson"), driver="GeoJSON")
point_whole_side_assigned.to_file(os.path.join(save_dir, "top_10_road_segments_v3.0_pano_32616.geojson"), driver="GeoJSON")

# Inspection

In [51]:
base_dir = "./output"

edge_whole_side_assigned = gpd.read_file(os.path.join(base_dir, "top_10_road_segments_v3.0.geojson"), driver="GeoJSON")
print(len(edge_whole_side_assigned))
#edge_whole_side_assigned.iloc[0:5]

point_whole_side_assigned = gpd.read_file(os.path.join(base_dir, "top_10_road_segments_v3.0_pano.geojson"), driver="GeoJSON")
print(len(point_whole_side_assigned))
point_whole_side_assigned.iloc[0:5]

/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
/home/jlee4338/anaconda3/envs/jlox/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


2746
17016


,link_id,side,point_id,osmid,median,bearing,pano_id,pano_lat,pano_lon,pano_heading,pano_date,is_midpoint,geometry
0,cluster10_primary_1_2,side1,0,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,L0BLSeHmcehpH9rKz6nM0Q,33.692660,-84.516929,303.943359,2024-06,False,POINT (-84.51693 33.69266)
1,cluster10_primary_1_2,side1,1,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,piW2JSC7o-e25SiBjEFtlw,33.692603,-84.516828,303.892731,2024-06,False,POINT (-84.51683 33.6926)
2,cluster10_primary_1_2,side1,2,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,X0Q1WyzMzxtIr2stSsZdQw,33.692546,-84.516726,303.913971,2024-06,True,POINT (-84.51673 33.69255)
3,cluster10_primary_1_2,side1,3,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,43EIDE2rVaHNv6xzSNywog,33.692488,-84.516623,304.025757,2024-06,False,POINT (-84.51662 33.69249)
4,cluster10_primary_1_2,side1,4,"[ 1145035104, 1145035105, 1145035106, 20726027...",no,122.951365,S03QTtGTPJyzG4B7XHUemQ,33.692430,-84.516521,304.231506,2024-06,False,POINT (-84.51652 33.69243)


In [55]:
link_ids = [
    "cluster15_tertiary_2_2",
    "cluster8_primary_6_6",
    "cluster8_secondary_12_3",
    "cluster8_secondary_4_2",
    "cluster8_secondary_4_4",
    "cluster8_secondary_4_6",
    "cluster8_secondary_6_3",
    "cluster8_tertiary_4_1",
    "cluster8_tertiary_6_2",
    "cluster8_tertiary_6_3",
    "cluster8_tertiary_6_4",
    "cluster8_tertiary_7_1"
]

pano_ids = [
    "CIHM0ogKEICAgIDL5q2d1gE",
    "CIHM0ogKEICAgIDLpu77vAE",
    "CIHM0ogKEICAgIDLpu77vAE",
    "CIHM0ogKEICAgIDLweC_0QE",
    "CIHM0ogKEICAgIDLqv3Jfg",
    "CIHM0ogKEICAgIDLqv3jmAE",
    "CIHM0ogKEICAgIDLpqu9Bw",
    "CIHM0ogKEICAgIDLprDPHQ",
    "CIHM0ogKEICAgIDLprDkNA",
    "CIHM0ogKEICAgIDLprCY7AE",
    "CIHM0ogKEICAgIDLprDkTA",
    "CIHM0ogKEICAgIDLpvCIWQ"
]

In [59]:
retrieve_error_links = point_whole_side_assigned[point_whole_side_assigned["link_id"].isin(link_ids)]
retrieve_error_panos = point_whole_side_assigned[point_whole_side_assigned["pano_id"].isin(pano_ids)]

In [62]:
m = retrieve_error_links.explore()

m = retrieve_error_panos.explore(
    m=m,
    color="red")
m

In [61]:
retrieve_error_links_cluster15 = point_whole_side_assigned[point_whole_side_assigned["link_id"].isin(["cluster15_tertiary_2_2"])]
retrieve_error_panos_cluster15 = point_whole_side_assigned[point_whole_side_assigned["pano_id"].isin(["CIHM0ogKEICAgIDL5q2d1gE"])]

m = retrieve_error_links_cluster15.explore()

m = retrieve_error_panos_cluster15.explore(
    m=m,
    color="red")
m